In [ ]:
!pip install rdflib
!pip install lxml
!pip install requests

In [1]:
# almost working

from rdflib import Graph, BNode, URIRef, Namespace, RDF
from name_space.alma_ns import alma_namespaces
import requests

In [2]:
instance_uri = "https://api.stage.sinopia.io/resource/c3905dd8-e349-49b4-9ad3-55baeae178eb"
# change the instance_uri to the URI of the instance that is linked to the work resource you want to serialize for Alma RDF/XML.
# add api key and region for posting the resource to Alma
# https://github.com/LD4P/ils-middleware/wiki/Alma-APIs-in-Airflow

alma_api_key = "your_api_key"
uri_region = "https://api-na.hosted.exlibrisgroup.com"

In [3]:
instance_uri = URIRef(instance_uri)
work_uri = None
instance_graph = Graph()
work_graph = Graph()
instance_graph.parse(instance_uri)

<Graph identifier=N58f6b6a841504cebbe116913e0b1061f (<class 'rdflib.graph.Graph'>)>

In [4]:
 # Define the bf and bflc namespaces
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")
for prefix, url in alma_namespaces:
    work_graph.bind(prefix, url)
work_uri = instance_graph.value(
    subject=URIRef(instance_uri), predicate=bf.instanceOf
        )
work_uri = URIRef(work_uri)
# Explicitly state that work_uri is of type bf:Work
work_graph.add((work_uri, RDF.type, bf.Work))

# parse the work graph
work_graph.parse(work_uri)

# add the instance to the work graph
work_graph.add((work_uri, bf.hasInstance, URIRef(instance_uri)))

<Graph identifier=Ne811e4c5254041a0a378f7268d5a38d5 (<class 'rdflib.graph.Graph'>)>

In [5]:
# Identify the blank nodes in the graph
bnodes = set()
for s, p, o in work_graph:
    if isinstance(s, BNode):
        bnodes.add(s)
    if isinstance(o, BNode):
        bnodes.add(o)

#  For each blank node, find the triples where it is used as a subject and as an object
for bnode in bnodes:
    subjects = [(s, p) for s, p in work_graph.subject_predicates(object=bnode)]
    objects = [(o, p) for p, o in work_graph.predicate_objects(subject=bnode)]

    # Step 4: Create new triples that directly connect the subjects and objects of these triples
    for (subject, pred1) in subjects:
        for (object, _) in objects:
            work_graph.add((subject, pred1, object))
                    
# add alma namespaces
for prefix, uri in alma_namespaces:
    work_graph.bind(prefix, Namespace(uri))

# serialize the graph
print(work_graph.serialize(format="pretty-xml"))

# save the graph to "pre-work.xml"
work_graph.serialize(destination="pre-work.xml", format="pretty-xml")



<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:bf="http://id.loc.gov/ontologies/bibframe/"
  xmlns:bflc="http://id.loc.gov/ontologies/bflc/"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:sinopia="http://sinopia.io/vocabulary/"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
>
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/085754a9-082b-476e-84b3-137425e3226a">
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="f58e01cd9010f40b491dd7df88a95098bb1">
        <bf:mainTitle xml:lang="en">Testing relations in Alma</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:title xml:lang="en">Testing relations in Alma</bf:title>
    <bf:title rdf:resource="http://id.loc.gov/ontologies/bibframe/Title"/>
    <bf:contribution>
      <bf:PrimaryContribution rdf:nodeID="f58e01cd9010f40b491dd7df88a95098bb2">
        <bf:agent rdf:resource="http://id.loc.gov/authorities/names/no

<Graph identifier=Ne811e4c5254041a0a378f7268d5a38d5 (<class 'rdflib.graph.Graph'>)>

In [6]:
from lxml import etree as ET

def remove_duplicates(parent):
    seen = set()
    for child in parent:
        if child.tag in seen and child.tag not in ['{http://id.loc.gov/ontologies/bibframe/}contribution', '{http://id.loc.gov/ontologies/bibframe/}note']:
            parent.remove(child)
        else:
            seen.add(child.tag)
        remove_duplicates(child)

# read the pre-work.xml file
tree = ET.parse("pre-work.xml")

# get the root
root = tree.getroot()

# Remove duplicates
remove_duplicates(root)

# save the xml to "work.xml"
with open("rel-work.xml", "wb") as f:
    f.write(ET.tostring(root, pretty_print=True))


In [7]:
 # serialize to xml
# open the work.xml file as bfwork_alma_xml
with open("rel-work.xml", "rb") as f:
    bfwork_alma_xml = f.read()
tree = ET.fromstring(bfwork_alma_xml)
# apply xslt to normalize instance
xslt = ET.parse("xsl/normalize-work-sinopia2alma.xsl")
transform = ET.XSLT(xslt)
bfwork_alma_xml = transform(tree)
bfwork_alma_xml = ET.tostring(
        bfwork_alma_xml, pretty_print=True, encoding="utf-8"
        )
# save the xml to a file
with open("rel-alma-work.xml", "wb") as f:
    f.write(bfwork_alma_xml)
print(bfwork_alma_xml.decode("utf-8"))

<bib>
  <record_format>lcbf_work</record_format>
  <suppress_from_publishing>false</suppress_from_publishing>
  <record>
    <rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/085754a9-082b-476e-84b3-137425e3226a" xmlns:bf="http://id.loc.gov/ontologies/bibframe/">
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="f58e01cd9010f40b491dd7df88a95098bb1">
        <bf:mainTitle xml:lang="en">Testing relations in Alma</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:contribution>
      <bf:PrimaryContribution rdf:nodeID="f58e01cd9010f40b491dd7df88a95098bb2">
        <bf:agent rdf:resource="http://id.loc.gov/authorities/names/no90020704"/>
   

POST to ALMA

In [8]:
# handle 400, an update to the record in Alma
def parse_400(result):
    xml_response = ET.fromstring(result)
    xslt = ET.parse("xsl/put_mms_id.xsl")
    transform = ET.XSLT(xslt)
    result_tree = transform(xml_response)
    put_mms_id_str = str(result_tree)
    print(f"put_mms_id_str: {put_mms_id_str}")
    return put_mms_id_str

# post the work to Alma
def NewWorktoAlma():
    with open("rel-alma-work.xml", "rb") as f:
        data = f.read()

        alma_uri = (
            uri_region
            + "/almaws/v1/bibs?"
            + "from_nz_mms_id=&from_cz_mms_id=&normalization=&validate=false"
            + "&override_warning=true&check_match=false&import_profile=&apikey="
            + alma_api_key
        )
        # post to alma
        alma_result = requests.post(
            alma_uri,
            headers={
                "Content-Type": "application/xml; charset=utf-8",
                "Accept": "application/xml",
                "x-api-key": alma_api_key,
            },
            data=data,
        )
        print(f"alma result: {alma_result.status_code}\n{alma_result.text}")
        result = alma_result.content
        status = alma_result.status_code
        if status == 200:
            xml_response = ET.fromstring(result)
            mms_id = xml_response.xpath("//mms_id/text()")
            print(f"Created record {mms_id}")
        elif status == 400:
            # run xslt on the result in case the response is 400 and we need to update the record
            put_mms_id_str = parse_400(result)
            alma_update_uri = (
                uri_region
                + "/almaws/v1/bibs/"
                + put_mms_id_str
                + "?normalization=&validate=false&override_warning=true"
                + "&override_lock=true&stale_version_check=false&cataloger_level=&check_match=false"
                + "&apikey="
                + alma_api_key
            )
            putWorkToAlma(
                alma_update_uri,
                data,
            )
        else:
            raise Exception(f"Unexpected status code from Alma API: {status}")

# update the instance in Alma
def putWorkToAlma(
    alma_update_uri,
    data,
):
    put_update = requests.put(
        alma_update_uri,
        headers={
            "Content-Type": "application/xml; charset=UTF-8",
            "Accept": "application/xml",
        },
        data=data,
    )
    print(f"put update: {put_update.status_code}\n{put_update.text}")
    put_update_status = put_update.status_code
    result = put_update.content
    xml_response = ET.fromstring(result)
    put_mms_id = xml_response.xpath("//mms_id/text()")
    match put_update_status:
        case 200:
            print(f"Updated record {put_mms_id}")
        case 500:
            raise Exception(f"Internal server error from Alma API: {put_update_status}")
        case _:
            raise Exception(
                f"Unexpected status code from Alma API: {put_update_status}"
            )

# Call the function
NewWorktoAlma()

alma result: 200
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><bib><mms_id>9779354624303681</mms_id><record_format>lc_bf_work</record_format><linked_record_id/><title>Testing relations in Alma</title><author>Doe, John,</author><holdings link="https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/9779354624303681/holdings"/><created_by>exl_api</created_by><created_date>2024-02-19Z</created_date><last_modified_by>exl_api</last_modified_by><last_modified_date>2024-02-19Z</last_modified_date><suppress_from_publishing>false</suppress_from_publishing><suppress_from_external_search>false</suppress_from_external_search><suppress_from_metadoor>false</suppress_from_metadoor><sync_with_oclc>NONE</sync_with_oclc><sync_with_libraries_australia>NONE</sync_with_libraries_australia><originating_system>01UPENN_INST</originating_system><originating_system_id>085754a9-082b-476e-84b3-137425e3226a</originating_system_id><record><rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xml